In [347]:
import scapy.all as sc
import numpy as np
import pandas as pd
from scipy.stats import entropy

In [349]:
SNIFF_TIMEOUT = 1200
BROADCAST = "BROADCAST"
UNICAST = "UNICAST"
PCAP_FILE = "packages.pcap"
ARP_OP_WHO_HAS = 1
ARP_OP_IS_AT = 2

## Hago sniff durante sniff_timeout segundos

In [350]:
packages = sc.sniff(timeout=SNIFF_TIMEOUT)

## Guardo los paquetes en formato pcap

In [351]:
sc.wrpcap(PCAP_FILE,packages)

## Leo los paquetes del archivo .pcap

In [352]:
packages = sc.sniff(offline=PCAP_FILE)

## Ejercicio 3.2.1

In [353]:
def simbolo_s(package):
    if p.dst == 'ff:ff:ff:ff:ff:ff':
        return BROADCAST
    else:
        return UNICAST

In [354]:
def entropia_fuente(fuente):
    counts = fuente.value_counts()
    return entropy(base=2,pk=counts.as_matrix())

In [355]:
def informacion_simbolos(fuente):
    counts = fuente.value_counts()
    total = counts.sum()
    probs = counts / float(total)
    return pd.Series(- np.log2(probs), index=counts.index)

In [356]:
fuente = pd.Series([simbolo_s(p) for p in packages])
contadores = fuente.value_counts()
print "Entropia fuente S =", entropia_fuente(fuente)
print contadores
print informacion_simbolos(fuente)

Entropia fuente S = 0.50597500931
UNICAST      6849
BROADCAST     864
dtype: int64
UNICAST      0.171399
BROADCAST    3.158189
dtype: float64


## Ejercicio 3.2.2

In [357]:
arp_packages = sc.sniff(offline=PCAP_FILE, lfilter=lambda x: x.haslayer(sc.ARP))

In [358]:
arp_packages = map(lambda p: p["ARP"], arp_packages)
who_has_packages = filter(lambda p : p.op == ARP_OP_WHO_HAS,arp_packages)
is_at_packages = filter(lambda p : not p.op == ARP_OP_WHO_HAS, arp_packages)

In [359]:
is_at_packages[0]

<ARP  hwtype=0x1 ptype=0x800 hwlen=6 plen=4 op=is-at hwsrc=50:b7:c3:06:2e:ec psrc=10.254.213.74 hwdst=00:e0:4c:b7:10:be pdst=10.254.213.84 |>

In [360]:
who_has_packages[0]

<ARP  hwtype=0x1 ptype=0x800 hwlen=6 plen=4 op=who-has hwsrc=c4:da:26:03:ba:a4 psrc=10.254.213.56 hwdst=00:00:00:00:00:00 pdst=10.254.213.254 |<Padding  load='\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' |>>

## Analisis de simbolos distinguidos utilizando who-has packages

In [361]:
def simbolo_s2(package):
    return package.pdst

In [362]:
fuente_s2 = pd.Series([simbolo_s2(p) for p in who_has_packages])
contadores = fuente_s2.value_counts()
print contadores
print "Entropia de la fuente =", entropia_fuente(fuente_s2)
print informacion_simbolos(fuente_s2)

10.254.213.254    23
10.254.213.74     10
10.254.95.254      5
10.254.213.45      2
10.254.213.8       2
10.254.213.25      1
10.254.213.57      1
dtype: int64
Entropia de la fuente = 1.98509420017
10.254.213.254    0.935870
10.254.213.74     2.137504
10.254.95.254     3.137504
10.254.213.45     4.459432
10.254.213.8      4.459432
10.254.213.25     5.459432
10.254.213.57     5.459432
dtype: float64


## Analisis de simbolos distinguidos utilizando is_at packages

In [363]:
def simbolo_s3(package):
    return package.psrc

In [364]:
fuente_s3 = pd.Series([simbolo_s3(p) for p in is_at_packages])
contadores_s3 = fuente_s3.value_counts()
print contadores_s3
print "Entropia de la fuente =", entropia_fuente(fuente_s3)
print informacion_simbolos(fuente_s3)

10.254.213.74    10
dtype: int64
Entropia de la fuente = -0.0
10.254.213.74   -0.0
dtype: float64


In [365]:
len(is_at_packages)

10